In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import warnings
import ast
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
credits = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')
movies = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')

In [ ]:
movies = pd.merge(movies,credits)

In [ ]:
movies.info()

In [ ]:
movies.head(1)

In [ ]:
movies.drop(['movie_id','homepage','budget','original_language','original_title','popularity','production_countries','revenue','runtime','spoken_languages','status','tagline','vote_average','vote_count','release_date'],axis=1,inplace=True)

In [ ]:
movies.isna().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
def convert(obj):
    L= [] 
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

movies['genres'] = movies.genres.apply(convert)

movies['keywords'] = movies.keywords.apply(convert)

In [ ]:
def convert(obj):
    L= [] 
    counter = 0
    for i in ast.literal_eval(obj):
        if counter<2:
            L.append(i['name'])
        counter+=1
    return L
movies['production_companies'] = movies.production_companies.apply(convert)

In [ ]:
def convertCast(obj):
    L= [] 
    counter = 0
    for i in ast.literal_eval(obj):
        if counter<3:
            L.append(i['name'])
        counter+=1
    return L
def convertCrew(obj):
    L= []
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
                L.append(i['name'])
    return L
movies['cast'] = movies.cast.apply(convertCast)
movies['crew'] = movies.crew.apply(convertCrew)

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['production_companies'] = movies['production_companies'].apply(lambda x:[i.replace(" ","") for i in x])

movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

movies['tag'] = movies['overview']+movies['genres']+movies['keywords']+movies['production_companies']+movies['cast']+movies['crew']

In [ ]:
new_df = movies[['id','title','tag']]
new_df['tag'] = new_df['tag'].apply(lambda x:" ".join(x))

In [ ]:
new_df['tag'] = new_df['tag'].apply(lambda x:x.lower())

In [ ]:
new_df[new_df['title']=='The Dark Knight']

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
new_df['tag'] = new_df['tag'].apply(stem)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tag']).toarray()

In [ ]:
vectors

In [ ]:
## corpos
cv.get_feature_names()

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
def recommend(movie):
    movie_indx = new_df[new_df['title']==movie].index[0]
    distances = similarity[movie_indx]  
    simiar_movies = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in simiar_movies:
        print (new_df.iloc[i[0]].id)
        print (new_df.iloc[i[0]].title)
recommend('Batman Begins')

In [ ]:
import pickle
pickle.dump(new_df,open('movies.pkl','wb'))
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))
pickle.dump(similarity,open('sim_matrix.pkl','wb'))